In [ ]:
class WindowGenerator():

    def __init__(self, input_width, label_width, shift,
                train_df, val_df, test_df,label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                                enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]
        
    def __repr__(self):
        return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])


    def split_window(self, features):   
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

    def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
        inputs, labels = self.example
        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        for n in range(max_n):
            plt.subplot(max_n, 1, n+1)
            plt.ylabel(f'{plot_col} [normed]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                    label='Inputs', marker='.', zorder=-10)

            if self.label_columns:
                label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
                label_col_index = plot_col_index

            if label_col_index is None:
                continue

            plt.scatter(self.label_indices, labels[n, :, label_col_index],
                        edgecolors='k', label='Labels', c='#2ca02c', s=64)
            if model is not None:
                predictions = model(inputs)
                plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                            marker='X', edgecolors='k', label='Predictions',
                            c='#ff7f0e', s=64)

            if n == 0:
                plt.legend()

        plt.xlabel('Time [h]')

    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32,)

        ds = ds.map(self.split_window)

        return ds

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        # """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
        # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
        # And cache it for next time
            self._example = result
        return result

In [ ]:
# ploting the weights and stuff

# plt.bar(x = range(len(train_df.columns)),
#         height=linear.layers[0].kernel[:,0].numpy())
# axis = plt.gca()
# axis.set_xticks(range(len(train_df.columns)))
# _ = axis.set_xticklabels(train_df.columns, rotation=90)

# lstm 
"""inputs = tf.random.normal([2, 3, 4])
lstm = tf.keras.layers.LSTM(units=5 ,return_sequences=True)

output = lstm(inputs)

output"""

In [ ]:
tf.keras.utils.plot_model(
    Dense_Model_3,
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=True,
    dpi=96,
    layer_range=None)

In [ ]:
testmodel = tf.keras.Sequential()
testmodel.add(LSTM(units=300, return_sequences=True , input_shape =(60,30)))
testmodel.add(Dropout(0.1))
testmodel.add(LSTM(units=720, return_sequences=True))
testmodel.add(Dropout(0.1))
testmodel.add(LSTM(units=360))
testmodel.add(Dense(units=30, activation='linear'))
testmodel.add(Reshape([10,3]))

history = compile_and_fit(testmodel,'testmodel',data,input_window,output_widow)
testmodel.save(f'models/testmodel/{data.stockname}/testmodel.h5')

predicted , real = data.test(testmodel , data.test_dataset)
data.plot_and_saveCSV(testmodel,'testmodel',predicted=predicted,real=real)

In [ ]:
dense_model_1 = tf.keras.Sequential(
    [
        Reshape((1,1800),input_shape=(60,30)),
        Dense(1800,activation='relu'),
        Dense(2400,activation='relu'),
        Dense(3000,activation='relu'),
        Dense(500,activation='relu'),
        Dense(30 ,activation='linear'),
        Reshape((10,3)),
    ]
)
history = compile_and_fit(dense_model_1,'dense_model_1',data,window_input=input_window,window_output=output_widow)
predicted , real = data.test(dense_model_1 , data.test_dataset)
data.plot_and_saveCSV(dense_model_1,'dense_model_1',predicted=predicted,real=real)


In [ ]:
dense_model_2 = Sequential()

dense_model_2.add(Dense(1800 , activation='relu',input_shape=(60,30)))
dense_model_2.add(Flatten())
dense_model_2.add(Dense(1200 , activation='relu'))
dense_model_2.add(Dense(600 , activation='relu'))
dense_model_2.add(Dense(300 , activation='relu'))
dense_model_2.add(Dense(200 , activation='relu'))
dense_model_2.add(Dense(300 , activation='relu'))
dense_model_2.add(Dense(400 , activation='relu'))
dense_model_2.add(Dense(500 , activation='relu'))
dense_model_2.add(Dense(600 , activation='relu'))
dense_model_2.add(Dense(1000 , activation='relu'))
dense_model_2.add(Dense(10 , activation='linear'))
dense_model_2.add(Reshape((10,1)))

dense_model_2.summary()
history = compile_and_fit(dense_model_2,'dense_model_2',data,window_input=input_window,window_output=output_widow)
predicted , real = data.test(dense_model_2 , data.test_dataset)
data.plot_and_saveCSV(dense_model_2,'dense_model_2',predicted=predicted,real=real)


In [ ]:
Dense_Model_3 = Sequential()

Dense_Model_3.add(Dense(30,activation=activations.linear,input_shape=(60,30),name='input'))
Dense_Model_3.add(Dense(100,activation=activations.softmax))
Dense_Model_3.add(Flatten())
Dense_Model_3.add(Dense(2000,activation=activations.softmax))
Dense_Model_3.add(Dense(2000,activation=activations.elu))
Dense_Model_3.add(Dense(3000,activation=activations.elu))
Dense_Model_3.add(Dense(4000,activation=activations.elu))
Dense_Model_3.add(Dense(5000,activation=activations.elu))
Dense_Model_3.add(Dense(6000,activation=activations.elu))
Dense_Model_3.add(Dense(7000,activation=activations.elu))
Dense_Model_3.add(Dense(10,activation=activations.linear))
Dense_Model_3.add(Reshape((10,1)))




Dense_Model_3.summary()
history = compile_and_fit(Dense_Model_3,'Dense_Model_3',data,window_input=input_window,window_output=output_widow)
predicted , real = data.test(Dense_Model_3 , data.test_dataset)
data.plot_and_saveCSV(Dense_Model_3,'Dense_Model_3',predicted=predicted,real=real)


In [ ]:
OUT_STEPS=10
num_features = 1
multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])




In [ ]:
num_features = input_window.shape[2]


class FeedBack(tf.keras.Model):
    def __init__(self, units, out_steps):
        super().__init__()
        self.out_steps = out_steps
        self.units = units
        self.lstm_cell = tf.keras.layers.LSTMCell(units)
        # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.dense = tf.keras.layers.Dense(num_features)

    def warmup(self, inputs):
        # inputs.shape => (batch, time, features)
        # x.shape => (batch, lstm_units)
        x, *state = self.lstm_rnn(inputs)

        # predictions.shape => (batch, features)
        prediction = self.dense(x)
        return prediction, state  

    def call(self, inputs, training=None):
        # Use a TensorArray to capture dynamically unrolled outputs.
        predictions = []
        # Initialize the LSTM state.
        prediction, state = self.warmup(inputs)

        # Insert the first prediction.
        predictions.append(prediction)

        # Run the rest of the prediction steps.
        for n in range(1, self.out_steps):
        # Use the last prediction as input.
            x = prediction
            # Execute one lstm step.
            x, state = self.lstm_cell(x, states=state,
                                        training=training)
            # Convert the lstm output to a prediction.
            prediction = self.dense(x)
            # Add the prediction to the output.
            predictions.append(prediction)

        # predictions.shape => (time, batch, features)
        predictions = tf.stack(predictions)
        # predictions.shape => (batch, time, features)
        predictions = tf.transpose(predictions, [1, 0, 2])
        return predictions  


feedback_model = FeedBack(units=32, out_steps=10)

history = compile_and_fit(feedback_model, 'feedback_model',data=data,window_input=input_window,window_output=output_widow)
# feedback_model.save(f'models/feedback_model/{data.stockname}/feedback_model')
predicted , real = data.test(feedback_model , data.test_dataset)
data.plot_and_saveCSV(feedback_model,'feedback_model',predicted=predicted,real=real)



In [10]:
x = iter(["apple", "banana", "cherry"])
x
print(next(x))


StopIteration: 